In [1]:
import polars as pl
import pandas as pd
import altair as alt
from pathlib import Path
from darts import TimeSeries

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [2]:
data_dir = Path("data")
df = pl.read_parquet(data_dir / "DSU-Dataset.parquet")

In [3]:
df = df.drop(
    "Claim ID", "Patient ID"
)  # we don't need these columns because they are unique identifiers are do not contain any useful information
df.describe()

statistic,Service Date,Recieved Date,Paid Date,Member Age,Gender,Marital Status,Ethnicity,LOB,Network Status,Claim Category,Claim Subcategory,Claim Line,Place of Service,Provider Type,Provider Specialty,ICD10 Code 1,ICD10 Code 2,ICD10 Code 3,ICD10 Code 4,ICD10 Code 5,ICD10 Code 6,ICD10 Code 7,ICD10 Code 8,ICD10 Code 9,ICD10 Code 10,Service Type,Service Code,Modifiers,High Cost Claim
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,f64
"""count""","""253686""","""253686""","""253686""","""253686""","""253686""","""241000""","""226800""","""253686""","""253686""","""253685""","""253685""","""253686""","""253686""","""253671""","""253685""",252750.0,176154.0,127472.0,95637.0,69465.0,56029.0,45262.0,37466.0,31514.0,26932.0,"""253686""","""253686""","""78847""",203250.0
"""null_count""","""0""","""0""","""0""","""0""","""0""","""12686""","""26886""","""0""","""0""","""1""","""1""","""0""","""0""","""15""","""1""",936.0,77532.0,126214.0,158049.0,184221.0,197657.0,208424.0,216220.0,222172.0,226754.0,"""0""","""0""","""174839""",50436.0
"""mean""","""2023-07-08 10:53:11.050000""","""2023-08-02 17:14:36.223000""","""2023-08-10 09:56:13.685000""",null,null,null,null,null,null,null,null,null,null,null,null,1458.673737,1357.81938,1332.740908,1424.046927,1446.2149,1533.74324,1541.26541,1650.423744,1555.836612,1755.552391,null,null,null,0.072315
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2224.261383,2053.669316,2098.782557,2170.225435,2130.954839,2224.144934,2267.625474,2383.157005,2272.372021,2408.165582,null,null,null,0.259009
"""min""","""2021-12-03""","""2021-12-06""","""2021-12-12""","""1-9 Yrs Old""","""F""","""Divorced""","""African American/Black""","""BES""","""IN NETWORK""","""HOSPICE""","""ACUTE INPATIENT""","""MEDICAL""","""AMBULANCE - AIR OR WATER""","""ACUPUNCTURIST""","""ACUPUNCTURIST""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""DRG""","""0001A""","""22""",0.0
"""25%""","""2022-10-08""","""2022-11-01""","""2022-11-09""",null,null,null,null,null,null,null,null,null,null,null,null,118.0,122.0,97.0,137.0,173.0,188.0,194.0,199.0,202.0,219.0,null,null,null,0.0
"""50%""","""2023-07-20""","""2023-08-14""","""2023-08-22""",null,null,null,null,null,null,null,null,null,null,null,null,539.0,541.0,486.0,519.0,580.0,622.0,629.0,741.0,622.0,796.0,null,null,null,0.0
"""75%""","""2024-04-15""","""2024-05-17""","""2024-05-25""",null,null,null,null,null,null,null,null,null,null,null,null,1696.0,1561.0,1467.0,1648.0,1673.0,1837.0,1776.0,1912.0,1742.0,2104.0,null,null,null,0.0
"""max""","""2025-01-05""","""2025-01-09""","""2025-01-12""","""< 1 Yrs Old""","""U""","""Widowed""","""Unavailable/Unknown""","""WRH""","""TIER 1""","""PROFESSIONAL""","""URGENT CARE""","""PHARMACY""","""WALK-IN RETAIL HEALTH CLINIC""","""Student Registered Nurse Anest…","""VASCULAR SURGERY (SURGERY)""",11323.0,11327.0,11316.0,11322.0,11271.0,11328.0,11325.0,11279.0,11212.0,11289.0,"""REV""","""V5275""","""XU,RT""",1.0


In [4]:
relevant_columns = [
    "Member Age",
    "Gender",
    "Marital Status",
    "Ethnicity",
    "LOB",
    "Network Status",
    "Claim Category",
    "Claim Subcategory",
    "Claim Line",
    "Place of Service",
    "Provider Type",
    "Provider Specialty",
    "Service Code",
    "Modifiers"
]

for column in relevant_columns:
    print(df.select(column).unique())

shape: (12, 1)
┌───────────────┐
│ Member Age    │
│ ---           │
│ str           │
╞═══════════════╡
│ 1-9 Yrs Old   │
│ 50-59 Yrs Old │
│ 10-19 Yrs Old │
│ 90-99 Yrs Old │
│ 30-39 Yrs Old │
│ …             │
│ < 1 Yrs Old   │
│ 80-89 Yrs Old │
│ 100+ Yrs Old  │
│ 40-49 Yrs Old │
│ 20-29 Yrs Old │
└───────────────┘
shape: (3, 1)
┌────────┐
│ Gender │
│ ---    │
│ str    │
╞════════╡
│ U      │
│ M      │
│ F      │
└────────┘
shape: (9, 1)
┌───────────────────┐
│ Marital Status    │
│ ---               │
│ str               │
╞═══════════════════╡
│ Married           │
│ Divorced          │
│ Partnered         │
│ Unknown           │
│ Legally Separated │
│ Widowed           │
│ null              │
│ Single            │
│ Other             │
└───────────────────┘
shape: (12, 1)
┌─────────────────────────────────┐
│ Ethnicity                       │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ Declined                 

In [5]:
df = df.filter(pl.col("High Cost Claim").is_not_null()) # remove rows with missing values in the target column

In [6]:
df.sample(10)

Service Date,Recieved Date,Paid Date,Member Age,Gender,Marital Status,Ethnicity,LOB,Network Status,Claim Category,Claim Subcategory,Claim Line,Place of Service,Provider Type,Provider Specialty,ICD10 Code 1,ICD10 Code 2,ICD10 Code 3,ICD10 Code 4,ICD10 Code 5,ICD10 Code 6,ICD10 Code 7,ICD10 Code 8,ICD10 Code 9,ICD10 Code 10,Service Type,Service Code,Modifiers,High Cost Claim
date,date,date,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,i64
2022-11-15,2022-11-21,2022-11-23,"""30-39 Yrs Old""","""F""","""Married""","""Caucasian/White""","""NPS""","""ND PERS PPO""","""PROFESSIONAL""","""OFFICE VISIT""","""PHARMACY""","""OFFICE""","""Physician""","""NP - FAMILY MEDICINE""",297,null,null,null,null,null,null,null,null,null,"""PROC""","""J1885""",null,0
2023-04-05,2023-04-27,2023-05-02,"""20-29 Yrs Old""","""F""","""Married""","""Caucasian/White""","""SGH""","""OUT OF NETWORK""","""PROFESSIONAL""","""OFFICE VISIT""","""MEDICAL""","""OFFICE""","""Physical Therapist""","""PHYSICAL THERAPIST""",816,null,null,null,null,null,null,null,null,null,"""PROC""","""97161""",null,0
2024-10-09,2024-10-26,2024-11-01,"""30-39 Yrs Old""","""F""","""Married""","""Caucasian/White""","""IET""","""IN NETWORK""","""PROFESSIONAL""","""OFFICE VISIT""","""MEDICAL""","""OFFICE""","""Physician Assistant""","""PA - ORTHOPEDIC SURGERY""",1303,406,148,320,257,0,null,null,null,null,"""PROC""","""80076""",null,0
2022-04-10,2022-04-17,2022-04-26,"""40-49 Yrs Old""","""F""","""Married""","""Caucasian/White""","""PIP""","""IN NETWORK""","""OUTPATIENT""","""OUTPATIENT""","""MEDICAL""","""ON CAMPUS - OUTPATIENT HOSPITA…","""Resource""","""ACUTE CARE HOSPITAL (GENERAL)""",737,null,null,null,null,null,null,null,null,null,"""PROC""","""82436""",null,0
2023-06-17,2023-06-18,2023-06-24,"""1-9 Yrs Old""","""M""","""Single""","""Caucasian/White""","""SOT""","""IN NETWORK""","""PROFESSIONAL""","""OFFICE VISIT""","""MEDICAL""","""OFFICE""","""Doctor Of Dental Surgery""","""DENTIST""",null,null,null,null,null,null,null,null,null,null,"""PROC""","""D1206""",null,0
2022-03-30,2022-04-05,2022-04-07,"""20-29 Yrs Old""","""M""","""Single""","""Caucasian/White""","""NPS""","""ND PERS PPO""","""PROFESSIONAL""","""OFFICE VISIT""","""MEDICAL""","""PUBLIC HEALTH CLINIC""","""Resource""","""ACUTE CARE HOSPITAL (GENERAL)""",5,null,null,null,null,null,null,null,null,null,"""PROC""","""90621""",null,0
2022-04-07,2022-05-02,2022-05-15,"""10-19 Yrs Old""","""F""","""Single""","""Caucasian/White""","""SOT""","""IN NETWORK""","""PROFESSIONAL""","""OFFICE VISIT""","""PHARMACY""","""OFFICE""","""Physician""","""PEDIATRIC HEMATOLOGY AND ONCOL…",3843,null,null,null,null,null,null,null,null,null,"""PROC""","""96375""",null,0
2022-06-25,2022-07-23,2022-07-28,"""60-69 Yrs Old""","""M""","""Married""","""Caucasian/White""","""SGH""","""IN NETWORK""","""OUTPATIENT""","""OUTPATIENT""","""MEDICAL""","""ON CAMPUS - OUTPATIENT HOSPITA…","""Resource""","""ACUTE CARE HOSPITAL (GENERAL)""",297,204,null,null,null,null,null,null,null,null,"""PROC""","""97140""","""GP""",0
2023-01-18,2023-01-24,2023-02-01,"""40-49 Yrs Old""","""M""","""Married""",null,"""IEX""","""IN NETWORK""","""PROFESSIONAL""","""OFFICE VISIT""","""MEDICAL""","""OFFICE""","""Physician Assistant""","""PHYSICIAN ASSISTANT""",1769,7188,7189,null,null,null,null,null,null,null,"""PROC""","""85025""",null,0


In [7]:
print(df.filter(pl.col("High Cost Claim") == 1).count())
print(df.filter(pl.col("High Cost Claim") == 0).count())

shape: (1, 29)
┌─────────┬─────────────────┬───────┬────────┬───┬─────────┬─────────┬───────────┬─────────────────┐
│ Service ┆ Recieved Date   ┆ Paid  ┆ Member ┆ … ┆ Service ┆ Service ┆ Modifiers ┆ High Cost Claim │
│ Date    ┆ ---             ┆ Date  ┆ Age    ┆   ┆ Type    ┆ Code    ┆ ---       ┆ ---             │
│ ---     ┆ u32             ┆ ---   ┆ ---    ┆   ┆ ---     ┆ ---     ┆ u32       ┆ u32             │
│ u32     ┆                 ┆ u32   ┆ u32    ┆   ┆ u32     ┆ u32     ┆           ┆                 │
╞═════════╪═════════════════╪═══════╪════════╪═══╪═════════╪═════════╪═══════════╪═════════════════╡
│ 14698   ┆ 14698           ┆ 14698 ┆ 14698  ┆ … ┆ 14698   ┆ 14698   ┆ 784       ┆ 14698           │
└─────────┴─────────────────┴───────┴────────┴───┴─────────┴─────────┴───────────┴─────────────────┘
shape: (1, 29)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Service   ┆ Recieved  ┆ Paid Date ┆ Member    ┆ … ┆ Service

In [8]:
time_claim = df.select("Service Date", "High Cost Claim")
time_claim.with_columns(time_claim["Service Date"].cast(pl.Date)).sort("Service Date")
time_claim = time_claim.group_by("Service Date").agg(pl.sum("High Cost Claim").alias("High Cost Claim"))
time_claim.plot.line(x="Service Date", y="High Cost Claim").properties(width=1200, height=600)

alt.Chart(...)

In [30]:
dummies_columns = [
    "Member Age",
    "Gender",
    "Marital Status",
    "Ethnicity",
    "LOB",
    "Network Status",
    "Claim Category",
    "Claim Subcategory",
    "Claim Line",
    "Place of Service",
    "Provider Type",
    "Provider Specialty",
]
df_dummies = df.drop("Modifiers", "Recieved Date", "Paid Date", "ICD10 Code 1", "ICD10 Code 2", "ICD10 Code 3", "ICD10 Code 4", "ICD10 Code 5", "ICD10 Code 6", "ICD10 Code 7", "ICD10 Code 8", "ICD10 Code 9", "ICD10 Code 10")
df_dummies = df_dummies.to_dummies(columns=dummies_columns)
df_dummies_pd = df_dummies.to_pandas()
df_dummies_timeseries = TimeSeries.from_dataframe(df_dummies_pd, time_col="Service Date", value_cols=["High Cost Claim"], freq="D")

c:\Users\Joe\Documents\DSU-Data-Competition-Spring2025\.venv\lib\site-packages\darts\timeseries.py:734: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  xa = xr.DataArray(


ValueError: cannot reindex on an axis with duplicate labels